# **Розширений план з контентом та прикладами коду для практичного заняття**
## **Тема 2. Заняття 10. Обробка інформації засобами бази даних**
### **1. Практична обробка інформації з використанням процедур та функцій**
### **2. Практична обробка інформації з використанням TCL (Transaction Control Language)**

---

## **1. Вступ (10 хв)**

### **1.1. Мета заняття**
- Ознайомити слухачів із використанням процедур та функцій у базах даних.
- Навчити застосовувати механізми управління транзакціями (TCL).
- Розглянути практичні приклади обробки інформації за допомогою SQL.

### **1.2. Очікувані результати**
Після заняття слухачі зможуть:
- Створювати збережені процедури (`Stored Procedures`) та функції (`Functions`).
- Використовувати `COMMIT`, `ROLLBACK` та `SAVEPOINT` для керування транзакціями.
- Автоматизувати обробку інформації в базах даних.

---

## **2. Практична обробка інформації з використанням процедур та функцій (60 хв)**

### **2.1. Основи використання процедур та функцій**
1. **Процедури** (`Stored Procedures`) – набір SQL-інструкцій, який виконується як одна логічна одиниця.
2. **Функції** (`Functions`) – повертають певне значення і можуть використовуватися у запитах.

---

### **2.2. Створення збережених процедур**
#### **2.2.1. Створення простої процедури**
Процедура, яка додає новий запис у таблицю `Equipment`:
```sql
CREATE OR REPLACE PROCEDURE add_equipment(
    p_name VARCHAR(100),
    p_type VARCHAR(50),
    p_status VARCHAR(50),
    p_unit_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO Equipment(name, type, status, unit_id)
    VALUES (p_name, p_type, p_status, p_unit_id);
END;
$$;
```
📌 **Виклик процедури**:
```sql
CALL add_equipment('Танк Т-90', 'Танк', 'Активна', 1);
```

---

#### **2.2.2. Процедура з умовами та перевірками**
Процедура, яка оновлює статус техніки, перевіряючи її наявність:
```sql
CREATE OR REPLACE PROCEDURE update_equipment_status(
    p_equipment_id INT,
    p_new_status VARCHAR(50)
)
LANGUAGE plpgsql
AS $$
BEGIN
    IF EXISTS (SELECT 1 FROM Equipment WHERE equipment_id = p_equipment_id) THEN
        UPDATE Equipment
        SET status = p_new_status
        WHERE equipment_id = p_equipment_id;
    ELSE
        RAISE NOTICE 'Техніка з ID % не знайдена', p_equipment_id;
    END IF;
END;
$$;
```
📌 **Виклик процедури**:
```sql
CALL update_equipment_status(3, 'На ремонті');
```

---

### **2.3. Створення функцій**
#### **2.3.1. Функція для підрахунку кількості техніки у підрозділі**
```sql
CREATE OR REPLACE FUNCTION count_equipment_in_unit(p_unit_id INT)
RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    v_count INT;
BEGIN
    SELECT COUNT(*) INTO v_count FROM Equipment WHERE unit_id = p_unit_id;
    RETURN v_count;
END;
$$;
```
📌 **Виклик функції**:
```sql
SELECT count_equipment_in_unit(1);
```

---

#### **2.3.2. Функція для отримання списку активної техніки**
```sql
CREATE OR REPLACE FUNCTION get_active_equipment()
RETURNS TABLE(equipment_id INT, name VARCHAR, type VARCHAR)
LANGUAGE plpgsql
AS $$
BEGIN
    RETURN QUERY SELECT equipment_id, name, type FROM Equipment WHERE status = 'Активна';
END;
$$;
```
📌 **Виклик функції**:
```sql
SELECT * FROM get_active_equipment();
```

---

## **3. Практична обробка інформації з використанням TCL (Transaction Control Language) (60 хв)**

### **3.1. Основи TCL**
1. **COMMIT** – фіксує всі зміни у поточній транзакції.
2. **ROLLBACK** – скасовує всі зміни у поточній транзакції.
3. **SAVEPOINT** – створює контрольну точку для часткового відновлення.

---

### **3.2. Використання COMMIT та ROLLBACK**
#### **3.2.1. Виконання транзакції з підтвердженням**
```sql
BEGIN;
INSERT INTO Equipment (name, type, status, unit_id) 
VALUES ('БТР-4', 'БТР', 'Активна', 2);
COMMIT;
```

#### **3.2.2. Виконання транзакції з відкатом**
```sql
BEGIN;
INSERT INTO Equipment (name, type, status, unit_id) 
VALUES ('Танк Т-80', 'Танк', 'На ремонті', 3);
ROLLBACK;
```
📌 **Результат**: дані про танк не будуть збережені.

---

### **3.3. Використання SAVEPOINT**
#### **3.3.1. Створення контрольної точки**
```sql
BEGIN;
INSERT INTO Equipment (name, type, status, unit_id) 
VALUES ('МАЗ-537', 'Тягач', 'Активна', 1);
SAVEPOINT sp1;

UPDATE Equipment 
SET status = 'На ремонті' 
WHERE name = 'МАЗ-537';
SAVEPOINT sp2;

ROLLBACK TO sp1; -- Відміна останнього оновлення
COMMIT; -- Фіксація змін
```
📌 **Результат**: техніка `МАЗ-537` буде додана, але її статус залишиться `Активна`.

---

## **4. Практичне завдання (30 хв)**

### **4.1. Завдання для слухачів**
1. **Створити процедуру, яка видаляє техніку за її ID.**
   - Використати `IF EXISTS`, щоб уникнути помилок.
2. **Розробити функцію, яка повертає кількість одиниць техніки певного типу.**
3. **Використати TCL для транзакційного внесення кількох змін.**
   - Додати нову техніку та змінити її статус.
   - Використати `SAVEPOINT` для часткового відновлення.

---

## **5. Висновки та підсумки заняття (20 хв)**

✅ **Основні висновки**:
- **Процедури та функції** спрощують роботу з базою даних та автоматизують процеси.
- **TCL (COMMIT, ROLLBACK, SAVEPOINT)** дозволяє керувати транзакціями для збереження або скасування змін.

✅ **Типові помилки та їх рішення**:
- ❌ **Відсутність перевірки існування даних перед оновленням** → 🔹 Використання `IF EXISTS`.
- ❌ **Незбережені зміни через відсутність COMMIT** → 🔹 Завжди перевіряти необхідність транзакції.

✅ **Домашнє завдання**:
- Створити **процедуру**, яка змінює статус усієї техніки певного підрозділу.
- Написати **функцію**, яка повертає список підрозділів із кількістю активної техніки.
- Використати **TCL**, щоб виконати оновлення декількох таблиць з можливістю відкату.

🚀 **Ключовий меседж**: Використання збережених процедур, функцій та механізмів управління транзакціями значно покращує ефективність роботи з базами даних та підвищує надійність інформаційно-аналітичних систем.